In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import random

# List of multiple realistic User-Agent strings
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/112.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
    'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36',
]

# Function to scrape rates from a given URL and return a DataFrame
def scrape_mortgage_rates(url, rate_column_name):
    # Select a random User-Agent
    headers = {
        'User-Agent': random.choice(user_agents)
    }
    
    # Fetch and parse HTML content with headers
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Extract table headers and data rows
    header = [th.get_text(strip=True) for th in soup.find('thead').find_all('th') if th.get_text(strip=True)]
    data_rows = []
    for row in soup.find_all('tbody'):
        for tr in row.find_all('tr'):
            columns = tr.find_all('td')
            if len(columns) >= 3:  # Ensure sufficient columns
                rate = columns[0].get_text(strip=True)
                provider = columns[1].get_text(strip=True)
                # Collect only necessary data (omit 'Payment' column)
                data_rows.append([rate, provider])

    # Create and return DataFrame with renamed rate column
    df = pd.DataFrame(data_rows, columns=header[:2])  # Only 'Rate' and 'Provider' are needed
    df.rename(columns={'Rate': rate_column_name}, inplace=True)
    return df

# URLs and column names for each mortgage type
urls = {
    'Variable rate 5y': 'https://www.ratehub.ca/best-mortgage-rates/5-year/variable',
    'Variable rate 3y': 'https://www.ratehub.ca/best-mortgage-rates/3-year/variable',
    'Fixed rate 1y': 'https://www.ratehub.ca/best-mortgage-rates/1-year/fixed',
    'Fixed rate 2y': 'https://www.ratehub.ca/best-mortgage-rates/2-year/fixed',
    'Fixed rate 3y': 'https://www.ratehub.ca/best-mortgage-rates/3-year/fixed',
    'Fixed rate 4y': 'https://www.ratehub.ca/best-mortgage-rates/4-year/fixed',
    'Fixed rate 5y': 'https://www.ratehub.ca/best-mortgage-rates/5-year/fixed'
}

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop through each URL, scrape data, and merge into result_df
for rate_name, url in urls.items():
    # Scrape data and merge
    df = scrape_mortgage_rates(url, rate_name)
    result_df = df if result_df.empty else result_df.merge(df, on='Provider', how='outer')
    
    # Wait for a random interval between 10-30 seconds
    time.sleep(random.uniform(10, 30))

# Final result contains all providers and their rates
result_df = result_df[['Provider'] + list(urls.keys())]

OurBestMortgageRates = result_df[:20].drop(columns=['Provider'])

# Convert percentages to float for comparison, processing each column individually
for col in OurBestMortgageRates.columns:
    OurBestMortgageRates[col] = OurBestMortgageRates[col].map(lambda x: float(x.strip('%')) if isinstance(x, str) else x)

# Set all values in each column to NaN except the minimum
for col in OurBestMortgageRates.columns:
    min_value = OurBestMortgageRates[col].min()
    OurBestMortgageRates[col] = OurBestMortgageRates[col].apply(lambda x: x if x == min_value else np.nan)

# Convert back to percentage strings
for col in OurBestMortgageRates.columns:
    OurBestMortgageRates[col] = OurBestMortgageRates[col].map(lambda x: f"{x}%" if pd.notnull(x) else "")

# Select only the first non-empty row per column
first_row_data = {col: OurBestMortgageRates[col][OurBestMortgageRates[col] != ""].iloc[0] for col in OurBestMortgageRates.columns}

# Convert the dictionary into a single-row DataFrame
OurBestMortgageRates = pd.DataFrame([first_row_data])

# Reset the index for clean display
OurBestMortgageRates = OurBestMortgageRates.T.reset_index()
OurBestMortgageRates.columns = ['Mortgage Type', 'Best Rate']

OurBestMortgageRates['Year'] = OurBestMortgageRates['Mortgage Type'].str.extract(r'(\d+\s*year|\d+y)').replace({'1y': '1 year', '2y': '2 year', '3y': '3 year', '4y': '4 year', '5y': '5 year'})
OurBestMortgageRates['Rate Type'] = OurBestMortgageRates['Mortgage Type'].apply(lambda x: 'Variable' if 'Variable' in x else 'Fixed')

# Pivot to get the desired columns
OurBestMortgageRates = OurBestMortgageRates.pivot(index='Year', columns='Rate Type', values='Best Rate').reset_index()

# Rename columns for clarity
OurBestMortgageRates.columns.name = None
OurBestMortgageRates = OurBestMortgageRates.rename(columns={'Year': 'TERMS', 'Fixed': 'FIXED', 'Variable': 'VARIABLE'})
OurBestMortgageRates.sort_values('TERMS', ascending=False, inplace=True)

In [2]:
OurBestMortgageRates

,TERMS,FIXED,VARIABLE
4,5 year,3.99%,4.7%
3,4 year,4.34%,NaN
2,3 year,4.19%,5.05%
1,2 year,5.34%,NaN
0,1 year,5.89%,NaN


In [3]:
result_df[:20]

,Provider,Variable rate 5y,Variable rate 3y,Fixed rate 1y,Fixed rate 2y,Fixed rate 3y,Fixed rate 4y,Fixed rate 5y
0,Canadian Lender,4.70%,5.05%,7.15%,5.49%,4.59%,4.89%,3.99%
1,CanwiseA Ratehub Company,4.95%,NaN,NaN,5.54%,4.44%,NaN,4.29%
2,Big 6 Bank,5.00%,NaN,5.89%,5.34%,4.39%,4.49%,4.44%
3,First National,5.00%,NaN,6.63%,5.92%,4.94%,4.89%,4.54%
4,CMLS Financial,5.05%,NaN,NaN,NaN,NaN,NaN,4.44%
5,CIBC,5.15%,5.45%,6.29%,5.59%,4.79%,4.64%,4.34%
6,Desjardins,5.20%,NaN,6.64%,5.64%,4.54%,4.34%,4.29%
7,MCAP,5.25%,NaN,7.44%,7.14%,5.09%,5.04%,4.74%
8,Scotiabank,5.25%,6.30%,6.24%,5.59%,4.64%,4.74%,4.69%
9,Alterna Savings,5.30%,5.30%,6.44%,6.34%,4.84%,5.54%,4.69%
